In [1]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

2023-08-02 15:38:06.043405: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 15:38:07.121660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')

ag_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model.to(device)
ag_pipeline = pipeline('sentiment-analysis', model=ag_model, tokenizer=ag_tokenizer)
ag_pipeline.device = next(ag_model.parameters()).device

ag_model_directory = "../../../models/bert-uncased_maskedlm_agnews_july31" #first diff
finetuned_ag_maskedlm = BertForMaskedLM.from_pretrained(ag_model_directory)
finetuned_ag_maskedlm.to(device)
ag_fill_mask = pipeline("fill-mask", model=finetuned_ag_maskedlm, tokenizer=ag_tokenizer)
ag_fill_mask.device = next(ag_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = TextFoolerJin2019

dataset_val = input('Enter the number of samples to run on (100 or 1000): ')

if dataset_val == '100':
    loaded_ag_100 = Dataset.load_from_disk('../data/filtered_ag_clean_100')
    ag_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_100))
    dataset = ag_100
    dataset_name = 'ag-news100'
elif dataset_val =='1000':
    loaded_ag_1000 = Dataset.load_from_disk('../data/filtered_ag_clean_1000')
    ag_1000 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_1000))
    dataset = ag_1000
    dataset_name = 'ag-news1000'
else:
    raise ValueError('Number of samples not supported')
    
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')
if defense == "default":
    ag_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(ag_model, ag_tokenizer)
    print(ag_wrapper)
elif defense == "logit":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(ag_wrapper)

cand_size = int(input('enter number of candidates (recommended 12 for quicker run, 50 otherwise): '))
# change candidate size
attack.transformation.max_candidates = cand_size
# adjust attack threshold to match Li et al. 2023 (0.5 theshold for AgNews Universal sentences encoder):
attack.constraints[2] = UniversalSentenceEncoder(metric = 'angular', threshold = 0.5, 
                                                 window_size = 15, skip_text_shorter_than_window=True, 
                                                 compare_against_original=False)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:4
Enter the number of samples to run on (100 or 1000): 1000
Specify a defense type among "default", "logit", "maj_log", "one_hot": one_hot
using num_voter = 11 and mask_pct = 0.3 with dataset = ag-news1000...
enter number of candidates (recommended 12 for quicker run, 50 otherwise): 12
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  12
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angu


  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2023-08-02 15:39:41.480704: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-02 15:39:43.388915: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]

  0%|          | 1/1000 [02:03<34:19:2


[Succeeded / Failed / Skipped / Total] 3 / 20 / 2 / 25:   3%|▎         | 26/1000 [50:36<31:35:59, 116.80s/it]
[Succeeded / Failed / Skipped / Total] 3 / 21 / 2 / 26:   3%|▎         | 26/1000 [50:36<31:36:00, 116.80s/it]
[Succeeded / Failed / Skipped / Total] 3 / 21 / 2 / 26:   3%|▎         | 27/1000 [51:06<30:42:03, 113.59s/it]
[Succeeded / Failed / Skipped / Total] 4 / 21 / 2 / 27:   3%|▎         | 27/1000 [51:06<30:42:04, 113.59s/it]
[Succeeded / Failed / Skipped / Total] 4 / 21 / 2 / 27:   3%|▎         | 28/1000 [52:38<30:27:35, 112.81s/it]
[Succeeded / Failed / Skipped / Total] 4 / 22 / 2 / 28:   3%|▎         | 28/1000 [52:38<30:27:36, 112.82s/it]
[Succeeded / Failed / Skipped / Total] 4 / 22 / 2 / 28:   3%|▎         | 29/1000 [52:39<29:23:04, 108.94s/it]
[Succeeded / Failed / Skipped / Total] 4 / 22 / 3 / 29:   3%|▎         | 29/1000 [52:39<29:23:04, 108.94s/it]
[Succeeded / Failed / Skipped / Total] 4 / 22 / 3 / 29:   3%|▎         | 30/1000 [54:00<29:06:24, 108.03s/it]
[Succeede


[Succeeded / Failed / Skipped / Total] 6 / 40 / 4 / 50:   5%|▌         | 51/1000 [1:30:31<28:04:23, 106.49s/it]
[Succeeded / Failed / Skipped / Total] 6 / 41 / 4 / 51:   5%|▌         | 51/1000 [1:30:31<28:04:23, 106.49s/it]
[Succeeded / Failed / Skipped / Total] 6 / 41 / 4 / 51:   5%|▌         | 52/1000 [1:33:16<28:20:20, 107.62s/it]
[Succeeded / Failed / Skipped / Total] 6 / 42 / 4 / 52:   5%|▌         | 52/1000 [1:33:16<28:20:20, 107.62s/it]
[Succeeded / Failed / Skipped / Total] 6 / 42 / 4 / 52:   5%|▌         | 53/1000 [1:35:32<28:27:02, 108.16s/it]
[Succeeded / Failed / Skipped / Total] 6 / 43 / 4 / 53:   5%|▌         | 53/1000 [1:35:32<28:27:03, 108.16s/it]
[Succeeded / Failed / Skipped / Total] 6 / 43 / 4 / 53:   5%|▌         | 54/1000 [1:38:00<28:36:51, 108.89s/it]
[Succeeded / Failed / Skipped / Total] 6 / 44 / 4 / 54:   5%|▌         | 54/1000 [1:38:00<28:36:51, 108.89s/it]
[Succeeded / Failed / Skipped / Total] 6 / 44 / 4 / 54:   6%|▌         | 55/1000 [1:40:32<28:47:35, 109


[Succeeded / Failed / Skipped / Total] 12 / 57 / 6 / 75:   8%|▊         | 76/1000 [2:16:09<27:35:19, 107.49s/it]
[Succeeded / Failed / Skipped / Total] 12 / 58 / 6 / 76:   8%|▊         | 76/1000 [2:16:09<27:35:19, 107.49s/it]
[Succeeded / Failed / Skipped / Total] 12 / 58 / 6 / 76:   8%|▊         | 77/1000 [2:18:18<27:37:57, 107.78s/it]
[Succeeded / Failed / Skipped / Total] 12 / 59 / 6 / 77:   8%|▊         | 77/1000 [2:18:18<27:37:57, 107.78s/it]
[Succeeded / Failed / Skipped / Total] 12 / 59 / 6 / 77:   8%|▊         | 78/1000 [2:20:50<27:44:47, 108.34s/it]
[Succeeded / Failed / Skipped / Total] 12 / 60 / 6 / 78:   8%|▊         | 78/1000 [2:20:50<27:44:47, 108.34s/it]
[Succeeded / Failed / Skipped / Total] 12 / 60 / 6 / 78:   8%|▊         | 79/1000 [2:22:43<27:43:54, 108.40s/it]
[Succeeded / Failed / Skipped / Total] 12 / 61 / 6 / 79:   8%|▊         | 79/1000 [2:22:43<27:43:54, 108.40s/it]
[Succeeded / Failed / Skipped / Total] 12 / 61 / 6 / 79:   8%|▊         | 80/1000 [2:25:12<27:4

...


[Succeeded / Failed / Skipped / Total] 93 / 793 / 39 / 925:  93%|█████████▎| 926/1000 [30:11:53<2:24:47, 117.40s/it]
[Succeeded / Failed / Skipped / Total] 93 / 794 / 39 / 926:  93%|█████████▎| 926/1000 [30:11:53<2:24:47, 117.40s/it]
[Succeeded / Failed / Skipped / Total] 93 / 794 / 39 / 926:  93%|█████████▎| 927/1000 [30:14:23<2:22:52, 117.44s/it]
[Succeeded / Failed / Skipped / Total] 93 / 795 / 39 / 927:  93%|█████████▎| 927/1000 [30:14:23<2:22:52, 117.44s/it]
[Succeeded / Failed / Skipped / Total] 93 / 795 / 39 / 927:  93%|█████████▎| 928/1000 [30:16:38<2:20:56, 117.46s/it]
[Succeeded / Failed / Skipped / Total] 93 / 796 / 39 / 928:  93%|█████████▎| 928/1000 [30:16:38<2:20:56, 117.46s/it]
[Succeeded / Failed / Skipped / Total] 93 / 796 / 39 / 928:  93%|█████████▎| 929/1000 [30:18:26<2:18:58, 117.44s/it]
[Succeeded / Failed / Skipped / Total] 93 / 797 / 39 / 929:  93%|█████████▎| 929/1000 [30:18:26<2:18:58, 117.44s/it]
[Succeeded / Failed / Skipped / Total] 93 / 797 / 39 / 929:  93


[Succeeded / Failed / Skipped / Total] 94 / 816 / 40 / 950:  95%|█████████▌| 951/1000 [31:00:03<1:35:50, 117.35s/it]
[Succeeded / Failed / Skipped / Total] 94 / 817 / 40 / 951:  95%|█████████▌| 951/1000 [31:00:04<1:35:50, 117.35s/it]
[Succeeded / Failed / Skipped / Total] 94 / 817 / 40 / 951:  95%|█████████▌| 952/1000 [31:02:58<1:33:55, 117.41s/it]
[Succeeded / Failed / Skipped / Total] 94 / 818 / 40 / 952:  95%|█████████▌| 952/1000 [31:02:58<1:33:55, 117.41s/it]
[Succeeded / Failed / Skipped / Total] 94 / 818 / 40 / 952:  95%|█████████▌| 953/1000 [31:04:18<1:31:56, 117.38s/it]
[Succeeded / Failed / Skipped / Total] 94 / 819 / 40 / 953:  95%|█████████▌| 953/1000 [31:04:18<1:31:56, 117.38s/it]
[Succeeded / Failed / Skipped / Total] 94 / 819 / 40 / 953:  95%|█████████▌| 954/1000 [31:04:45<1:29:54, 117.28s/it]
[Succeeded / Failed / Skipped / Total] 95 / 819 / 40 / 954:  95%|█████████▌| 954/1000 [31:04:45<1:29:54, 117.28s/it]
[Succeeded / Failed / Skipped / Total] 95 / 819 / 40 / 954:  96


[Succeeded / Failed / Skipped / Total] 98 / 836 / 41 / 975:  98%|█████████▊| 976/1000 [31:40:06<46:43, 116.81s/it]
[Succeeded / Failed / Skipped / Total] 98 / 837 / 41 / 976:  98%|█████████▊| 976/1000 [31:40:06<46:43, 116.81s/it]
[Succeeded / Failed / Skipped / Total] 98 / 837 / 41 / 976:  98%|█████████▊| 977/1000 [31:42:12<44:46, 116.82s/it]
[Succeeded / Failed / Skipped / Total] 98 / 838 / 41 / 977:  98%|█████████▊| 977/1000 [31:42:12<44:46, 116.82s/it]
[Succeeded / Failed / Skipped / Total] 98 / 838 / 41 / 977:  98%|█████████▊| 978/1000 [31:44:19<42:50, 116.83s/it]
[Succeeded / Failed / Skipped / Total] 98 / 839 / 41 / 978:  98%|█████████▊| 978/1000 [31:44:19<42:50, 116.83s/it]
[Succeeded / Failed / Skipped / Total] 98 / 839 / 41 / 978:  98%|█████████▊| 979/1000 [31:44:19<40:50, 116.71s/it]
[Succeeded / Failed / Skipped / Total] 98 / 839 / 42 / 979:  98%|█████████▊| 979/1000 [31:44:19<40:50, 116.71s/it]
[Succeeded / Failed / Skipped / Total] 98 / 839 / 42 / 979:  98%|█████████▊| 98

[Succeeded / Failed / Skipped / Total] 100 / 858 / 42 / 1000: 100%|██████████| 1000/1000 [32:23:22<00:00, 116.60s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 100    |
| Number of failed attacks:     | 858    |
| Number of skipped attacks:    | 42     |
| Original accuracy:            | 95.8%  |
| Accuracy under attack:        | 85.8%  |
| Attack success rate:          | 10.44% |
| Average perturbed word %:     | 3.07%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 235.43 |
+-------------------------------+--------+
The above are results for TextFoolerJin2019_ag-news1000_candsize12_mp0.3_nv11_one_hot.
